이 실습은 Runpod에서 A100 SXM GPU 1개로 진행되었습니다.

## 1. 필요한 패키지 설치

In [1]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. 데이터 전처리

In [2]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

1. **JSON 파일 로드 함수 정의**
- `load_dataset()` 함수를 정의하여 지정된 파일 경로에서 JSON 파일을 읽어옵니다. `open()` 함수에 `encoding='utf-8'` 옵션을 사용하여 한글 등 유니코드 문자가 포함된 데이터도 올바르게 처리할 수 있도록 합니다. `json.load()`를 통해 JSON 형식의 데이터를 파이썬 객체로 변환하여 반환합니다.

2. **train/test 분할 비율 설정**
- `test_ratio = 0.5`로 테스트 분할 비율을 50%로 설정합니다. 이는 전체 데이터 중 절반을 테스트용으로, 나머지 절반을 학습용으로 사용하겠다는 의미입니다. 이 비율은 필요에 따라 조정 가능하며, 일반적으로 0.2(20%) 또는 0.3(30%)으로 설정하기도 합니다.

3. **데이터 로드 및 크기 확인**
- `load_dataset('text_to_sql_data.json')`을 호출하여 text-to-sql 데이터셋을 메모리로 로드합니다. 이 데이터셋은 instruction과 output 필드로 구성되어 있으며, instruction은 자연어 질문을, output은 해당하는 SQL 쿼리를 담고 있습니다.
- `print(f"전체 데이터 개수: {len(dataset)}")`로 로드된 데이터셋의 총 샘플 수를 출력하여 데이터 규모를 확인합니다.

4. **전체 데이터의 인덱스를 train/test로 분할**
- `data_indices = list(range(len(dataset)))`로 0부터 데이터셋 길이-1까지의 연속된 인덱스 리스트를 생성합니다.
- `random.shuffle(data_indices)`를 사용하여 인덱스 리스트를 무작위로 섞습니다. 이는 데이터의 원래 순서에 따른 편향을 방지하고, 훈련 및 테스트 데이터가 고르게 분포되도록 보장합니다.
- `test_size = int(len(data_indices) * test_ratio)`로 테스트 데이터셋에 포함될 샘플 수를 계산합니다.
- `test_indices = data_indices[:test_size]`로 섞인 인덱스 리스트의 앞부분을 테스트 데이터 인덱스로 할당합니다.
- `train_indices = data_indices[test_size:]`로 나머지 인덱스들을 훈련 데이터 인덱스로 할당합니다.
- 각각의 인덱스 개수를 출력하여 분할이 올바르게 수행되었는지 확인합니다.

5. **OpenAI format으로 데이터 변환을 위한 함수**
- `format_data()` 함수를 정의하여 각 샘플을 OpenAI API와 호환되는 메시지 형식으로 변환합니다.
- 이 함수는 입력된 샘플에서 "instruction"과 "output" 필드를 추출하여 "messages" 배열을 구성합니다.
- 시스템 메시지에는 "당신은 text-to-sql을 수행해야 합니다."라는 역할 정의를 포함하여 AI 모델에게 수행할 작업을 명확히 지시합니다.
- 사용자 메시지에는 원본 데이터의 "instruction" 필드(자연어 질문)를 배치합니다.
- 어시스턴트 응답에는 "output" 필드(해당하는 SQL 쿼리)를 배치합니다.
- 반환되는 형식은 OpenAI의 chat completion API와 호환되는 형태로, 대화형 AI 훈련에 바로 사용할 수 있습니다.

6. **분할된 데이터를 OpenAI format으로 변환**
- `train_dataset = [format_data(dataset[i]) for i in train_indices]`로 훈련용 인덱스에 해당하는 모든 데이터를 OpenAI 형식으로 변환합니다.
- `test_dataset = [format_data(dataset[i]) for i in test_indices]`로 테스트용 인덱스에 해당하는 모든 데이터를 OpenAI 형식으로 변환합니다.
- 리스트 컴프리헨션을 사용하여 효율적으로 대량의 데이터를 일괄 변환하며, 각 샘플은 메시지 형태의 대화 구조로 재구성됩니다.
- 변환된 데이터는 파인튜닝이나 프롬프트 엔지니어링에 즉시 활용할 수 있는 표준화된 형식을 갖게 됩니다.

In [3]:
!wget https://github.com/llm-fine-tuning/LLaMA-Factory/raw/refs/heads/main/data/text_to_sql_data.json

--2025-05-31 18:18:26--  https://github.com/llm-fine-tuning/LLaMA-Factory/raw/refs/heads/main/data/text_to_sql_data.json
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/text_to_sql_data.json [following]
--2025-05-31 18:18:27--  https://raw.githubusercontent.com/llm-fine-tuning/LLaMA-Factory/refs/heads/main/data/text_to_sql_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3676178 (3.5M) [application/octet-stream]
Saving to: ‘text_to_sql_data.json.1’

text_to_sql_data.js 100%[===================>]   3.51M  --.-KB/s    in 0.03s   

2025-05

In [4]:
import json
import random

# 1. JSON 파일 로드
def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    return dataset

# 2. train/test 분할 비율 설정 (0.5면 5:5로 분할)
test_ratio = 0.5

# 3. 데이터 로드
dataset = load_dataset('text_to_sql_data.json')
print(f"전체 데이터 개수: {len(dataset)}")

전체 데이터 개수: 5000


In [5]:
# 4. 전체 데이터의 인덱스를 train/test로 분할
data_indices = list(range(len(dataset)))
random.shuffle(data_indices)  # 랜덤 셔플로 더 좋은 분할

test_size = int(len(data_indices) * test_ratio)
test_indices = data_indices[:test_size]
train_indices = data_indices[test_size:]

print(f"Train 인덱스 개수: {len(train_indices)}")
print(f"Test 인덱스 개수: {len(test_indices)}")

Train 인덱스 개수: 2500
Test 인덱스 개수: 2500


In [6]:
# 5. OpenAI format으로 데이터 변환을 위한 함수
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": "당신은 text-to-sql을 수행해야 합니다."
            },
            {
                "role": "user", 
                "content": sample["instruction"]  # instruction을 user prompt로 사용
            },
            {
                "role": "assistant",
                "content": sample["output"]  # output을 assistant response로 사용
            }
        ]
    }

# 6. 분할된 데이터를 OpenAI format으로 변환
train_dataset = [format_data(dataset[i]) for i in train_indices]
test_dataset = [format_data(dataset[i]) for i in test_indices]

In [7]:
# 임의의 345번 데이터 출력
train_dataset[345]["messages"]

[{'role': 'system', 'content': '당신은 text-to-sql을 수행해야 합니다.'},
 {'role': 'user',
  'content': "입력 텍스트: 보안 사건에 대한 평균 반응 시간이 가장 긴 사용자는 누구인가요?\n\nDDL statements:\nCREATE TABLE user_reactions (id INT, user_id INT, incident_id INT, reaction_time INT); CREATE TABLE users (id INT, name VARCHAR(50)); INSERT INTO user_reactions (id, user_id, incident_id, reaction_time) VALUES (1, 1, 1, 60), (2, 2, 2, 30), (3, 3, 3, 90), (4, 1, 4, 120), (5, 4, 5, 45), (6, 5, 6, 75), (7, 2, 7, 105); INSERT INTO users (id, name) VALUES (1, 'Alice'), (2, 'Bob'), (3, 'Charlie'), (4, 'David'), (5, 'Eve');\n\n위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다."},
 {'role': 'assistant',
  'content': '쿼리 작성: SELECT users.name, AVG(user_reactions.reaction_time) as avg_reaction_time FROM user_reactions INNER JOIN users ON user_reactions.user_id = users.id GROUP BY users.name ORDER BY avg_reaction_time DESC;'}]

In [8]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)
print(type(train_dataset))
print(type(test_dataset))

<class 'list'>
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [9]:
train_dataset[0]

{'messages': [{'content': '당신은 text-to-sql을 수행해야 합니다.', 'role': 'system'},
  {'content': '입력 텍스트: 2021년 3분기에 EU에서 판매된 지속 가능성 점수가 7점 미만인 제품을 제외한 스킨케어 제품의 평균 지속 가능성 점수를 계산하세요.\n\nDDL statements:\nCREATE TABLE skincare_sales(sale_id INT, product_id INT, sale_date DATE, sustainability_score INT);CREATE TABLE products(product_id INT, product_name TEXT, category TEXT, country TEXT);\n\n위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.',
   'role': 'user'},
  {'content': "쿼리 작성: SELECT AVG(s.sustainability_score) FROM skincare_sales s JOIN products p ON s.product_id = p.product_id WHERE p.category = 'skincare' AND p.country LIKE 'EU%' AND s.sustainability_score >= 7 AND DATE_PART('year', s.sale_date) = 2021 AND DATE_PART('quarter', s.sale_date) = 3;",
   'role': 'assistant'}]}

## 2. 모델 로드 및 템플릿 적용

In [10]:
# 허깅페이스 모델 ID
model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [11]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 text-to-sql을 수행해야 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력 텍스트: 2021년 3분기에 EU에서 판매된 지속 가능성 점수가 7점 미만인 제품을 제외한 스킨케어 제품의 평균 지속 가능성 점수를 계산하세요.

DDL statements:
CREATE TABLE skincare_sales(sale_id INT, product_id INT, sale_date DATE, sustainability_score INT);CREATE TABLE products(product_id INT, product_name TEXT, category TEXT, country TEXT);

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

쿼리 작성: SELECT AVG(s.sustainability_score) FROM skincare_sales s JOIN products p ON s.product_id = p.product_id WHERE p.category = 'skincare' AND p.country LIKE 'EU%' AND s.sustainability_score >= 7 AND DATE_PART('year', s.sale_date) = 2021 AND DATE_PART('quarter', s.sale_date) = 3;<|eot_id|>


## 3. LoRA와 SFTConfig 설정

In [12]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

`lora_alpha`: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.

`lora_dropout`: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1로 설정하면 학습 중 10%의 뉴런이 비활성화됩니다.

`r`: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.

`bias`: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. "none"으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. "all" 또는 "lora_only"와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.

`target_modules`: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 "q_proj"와 "v_proj"를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.

`task_type:` LoRA가 적용되는 작업 유형을 지정합니다. "CAUSAL_LM"은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 "SEQ2SEQ_LM"(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [13]:
args = SFTConfig(
    output_dir="llama3-8b-text-to-sql",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=2,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

`utput_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

`num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니다.

`per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있습니다.

`gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 2로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 유용합니다.

`gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질 수 있습니다.

`optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적화기를 사용합니다.

`logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마다 로그를 기록합니다.

`save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스텝마다 모델이 저장됩니다.

`save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50 스텝마다 모델을 저장합니다.

`bf16`: bfloat16 정밀도를 사용하도록 설정합니다. bfloat16은 FP32와 유사한 범위를 제공하면서 메모리와 계산 효율성을 높입니다.

`learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 사용하여 안정적인 학습을 유도합니다.

`max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 조정하여 폭발적 그래디언트를 방지합니다.

`warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다. 학습의 안정성을 높이기 위해 사용됩니다.

`lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"constant"`는 학습률을 일정하게 유지합니다.

`push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니다. `False`로 설정하면 업로드하지 않습니다.

`remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다. True로 설정하면 메모리를 절약할 수 있습니다.

`dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset: True`로 설정하여 데이터셋 준비 단계를 건너뜁니다.

`report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn

In [15]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        messages = example["messages"]

        # LLaMA 3 채팅 템플릿 적용 (시작 토큰 포함)
        prompt = "<|begin_of_text|>"
        for msg in messages:
            role = msg["role"]
            content = msg["content"].strip()
            prompt += f"<|start_header_id|>{role}<|end_header_id|>\n{content}<|eot_id|>"

        # 마지막 assistant 메시지는 응답으로 간주하고 레이블에 포함
        text = prompt.strip()

        # 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        labels = [-100] * len(input_ids)

        # assistant 응답의 시작 위치 찾기
        assistant_header = "<|start_header_id|>assistant<|end_header_id|>\n"
        assistant_tokens = tokenizer.encode(assistant_header, add_special_tokens=False)
        eot_token = "<|eot_id|>"
        eot_tokens = tokenizer.encode(eot_token, add_special_tokens=False)

        # 레이블 범위 지정
        i = 0
        while i <= len(input_ids) - len(assistant_tokens):
            if input_ids[i:i + len(assistant_tokens)] == assistant_tokens:
                start = i + len(assistant_tokens)
                end = start
                while end <= len(input_ids) - len(eot_tokens):
                    if input_ids[end:end + len(eot_tokens)] == eot_tokens:
                        break
                    end += 1
                for j in range(start, end):
                    labels[j] = input_ids[j]
                for j in range(end, end + len(eot_tokens)):
                    labels[j] = input_ids[j]  # <|eot_id|> 토큰도 포함
                break
            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 처리
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    for i in range(len(new_batch["input_ids"])):
        pad_len = max_length - len(new_batch["input_ids"][i])
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * pad_len)
        new_batch["attention_mask"][i].extend([0] * pad_len)
        new_batch["labels"][i].extend([-100] * pad_len)

    for k in new_batch:
        new_batch[k] = torch.tensor(new_batch[k])

    return new_batch

입력으로 사용되는 라마 챗 템플릿은 아래와 같습니다.되어 반환됩니다.

```python
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

시스템 프롬프트<|eot_id|><|start_header_id|>user<|end_header_id|>

유저 프롬프트<|eot_id|><|start_header_id|>assistant<|end_header|>LLM의 답변<|eot_id|>
```

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 label_ids를 생성합니다. 레이블 데이터의 경우 실제 assistant 응답 부분을 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

In [16]:
# 데이터의 최대 길이 제한
max_seq_length=8192

# collate_fn 테스트 (배치 크기 1. 즉, 데이터 1개에 대해서 전처리를 진행해본다.)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 227])
어텐션 마스크 형태: torch.Size([1, 227])
레이블 형태: torch.Size([1, 227])


In [17]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[128000, 128000, 128006, 9125, 128007, 198, 65895, 83628, 34804, 1495, 4791, 1355, 1498, 18359, 125201, 110513, 109670, 13, 128009, 128006, 882, 128007, 198, 44966, 29854, 10997, 45204, 54289, 25, 220, 2366, 16, 100392, 220, 18, 80816, 109509, 10013, 57575, 116604, 53400, 67890, 102130, 96451, 33931, 106313, 123503, 220, 22, 101838, 101412, 73653, 32428, 112785, 18359, 63171, 104065, 24486, 80307, 45780, 61857, 107213, 32179, 112785, 21028, 105276, 67890, 102130, 96451, 33931, 106313, 120045, 95303, 86157, 92245, 382, 59881, 12518, 512, 23421, 14700, 80705, 48167, 1161, 1604, 851, 9403, 11, 2027, 851, 9403, 11, 6412, 4257, 29643, 11, 41329, 10622, 9403, 1237, 23421, 14700, 3956, 20040, 851, 9403, 11, 2027, 1292, 16139, 11, 5699, 16139, 11, 3224, 16139, 629, 82001, 21028, 107573, 13094, 105551, 104167, 42529, 81673, 41820, 110257, 43449, 10997, 45204, 54289, 18918, 82818, 120378, 43139, 8029, 3396, 123, 120, 106064, 114839, 61938, 13, 128009, 128006, 78191, 128007, 198

In [18]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 text-to-sql을 수행해야 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>
입력 텍스트: 2021년 3분기에 EU에서 판매된 지속 가능성 점수가 7점 미만인 제품을 제외한 스킨케어 제품의 평균 지속 가능성 점수를 계산하세요.

DDL statements:
CREATE TABLE skincare_sales(sale_id INT, product_id INT, sale_date DATE, sustainability_score INT);CREATE TABLE products(product_id INT, product_name TEXT, category TEXT, country TEXT);

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
쿼리 작성: SELECT AVG(s.sustainability_score) FROM skincare_sales s JOIN products p ON s.product_id = p.product_id WHERE p.category = 'skincare' AND p.country LIKE 'EU%' AND s.sustainability_score >= 7 AND DATE_PART('year', s.sale_date) = 2021 AND DATE_PART('quarter', s.sale_date) = 3;<|eot_id|>


In [19]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 115740, 120, 29102, 114839, 25, 19638, 71514, 1161, 516, 74585, 10622, 8, 4393, 80705, 48167

In [20]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):
쿼리 작성: SELECT AVG(s.sustainability_score) FROM skincare_sales s JOIN products p ON s.product_id = p.product_id WHERE p.category = 'skincare' AND p.country LIKE 'EU%' AND s.sustainability_score >= 7 AND DATE_PART('year', s.sale_date) = 2021 AND DATE_PART('quarter', s.sale_date) = 3;<|eot_id|>


## 5. 어텐션 마스크 확인

### 배치 크기의 의미

배치 크기란 모델이 한 번에 학습하는 데이터 샘플의 수를 의미합니다. 예를 들어 배치 크기가 3이면, 모델은 세 개의 데이터 샘플을 동시에 처리합니다. 이렇게 병렬적으로 학습하면 계산 효율성이 높아지고 학습 속도가 빨라지는 이점이 있습니다.

### 텍스트 데이터의 길이 문제

자연어 처리에서 각 샘플(문장, 대화 등)은 길이가 다양합니다. 예를 들어 배치 크기 3인 경우:

- 샘플1: "인공지능이란 무엇인가요?" → [101, 4089, 8024, 6356, 102] (5 토큰)
- 샘플2: "오늘 날씨가 정말 좋네요." → [101, 3157, 2533, 4120, 2642, 8730, 6824, 102] (8 토큰)
- 샘플3: "딥러닝 모델을 학습시키는 방법을 알려주세요." → [101, 2982, 3478, 4567, 2053, 8276, 5036, 2355, 4602, 7312, 102] (11 토큰)

여기서 101과 102는 특수 토큰으로, 각각 문장의 시작과 끝을 표시합니다.

### 패딩의 필요성

신경망의 내부 연산은 고정된 크기의 입력을 요구합니다. 이 문제를 해결하기 위해 '패딩'을 사용하여 모든 샘플의 길이를 배치 내 가장 긴 샘플에 맞춥니다.

위 예시에서는 가장 긴 샘플3(11 토큰)에 맞춰 다른 샘플들에 패딩(0)을 추가합니다:

- 샘플1: [101, 4089, 8024, 6356, 102, 0, 0, 0, 0, 0, 0] (5 실제 + 6 패딩)
- 샘플2: [101, 3157, 2533, 4120, 2642, 8730, 6824, 102, 0, 0, 0] (8 실제 + 3 패딩)
- 샘플3: [101, 2982, 3478, 4567, 2053, 8276, 5036, 2355, 4602, 7312, 102] (11 실제 토큰)

이렇게 하면 모든 샘플이 동일한 길이(11)를 가지게 되어 하나의 배치로 처리할 수 있습니다.

### 어텐션 마스크의 필요성

패딩을 추가하면 모델이 어떤 토큰이 실제 내용이고 어떤 토큰이 의미 없는 패딩인지 구분해야 합니다. 이를 위해 '어텐션 마스크'를 사용합니다:

- 샘플1 마스크: [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
- 샘플2 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
- 샘플3 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

트랜스포머 모델의 어텐션 메커니즘은 이 마스크를 사용하여 패딩 토큰을 무시하고 실제 의미 있는 토큰에만 집중합니다. 이렇게 하면 패딩된 부분이 모델의 예측이나 학습에 영향을 미치지 않게 됩니다.

In [21]:
# 데이터의 최대 길이 제한
max_seq_length = 8192
# 0번과 1번 데이터의 길이 확인
example0 = train_dataset[0]
example1 = train_dataset[1]
# 개별 길이 확인 (토큰화 후)
tokenized0 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example0["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
tokenized1 = tokenizer(
    # 전체 처리 과정과 동일하게 전체 대화를 토큰화
    "<|begin_of_text|>" + "".join([f"<|start_header_id|>{msg['role']}<|end_header_id|>\n{msg['content'].strip()}<|eot_id|>" for msg in example1["messages"]]),
    truncation=True,
    max_length=max_seq_length,
    padding=False,
    return_tensors=None,
)
print(f"0번 데이터 길이: {len(tokenized0['input_ids'])}")
print(f"1번 데이터 길이: {len(tokenized1['input_ids'])}")
# 배치로 처리하여 어텐션 마스크 비교
batch = collate_fn([example0, example1])
print("\n배치 처리 후:")
print(f"입력 ID 형태: {batch['input_ids'].shape}")
print(f"어텐션 마스크 형태: {batch['attention_mask'].shape}")
# 각 샘플의 어텐션 마스크 합계 (실제 토큰 수 확인)
print(f"0번 샘플 어텐션 마스크 합계: {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계: {batch['attention_mask'][1].sum().item()}")
# 0번 샘플과 1번 샘플의 어텐션 마스크가 다른지 확인
masks_different = not torch.equal(batch['attention_mask'][0], batch['attention_mask'][1])
print(f"\n0번과 1번 샘플의 어텐션 마스크가 다른가요? {masks_different}")
# 어텐션 마스크 패턴 시각화 (처음 20개와 마지막 20개 토큰)
print("\n0번 샘플 어텐션 마스크:", batch['attention_mask'][0].tolist())
print("1번 샘플 어텐션 마스크:", batch['attention_mask'][1].tolist())
# 배치 내에서 가장 긴 시퀀스 길이 구하기
max_length_in_batch = max(len(tokenized0['input_ids']), len(tokenized1['input_ids']))
print(f"\n배치 내 최대 길이: {max_length_in_batch}")
print(f"0번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][0].sum().item()}")
print(f"1번 샘플 어텐션 마스크 합계 (실제 토큰 수): {batch['attention_mask'][1].sum().item()}")
print(f"0번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][0].sum().item()}")
print(f"0번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][0] == 0).sum().item()}")
print(f"1번 샘플 어텐션 마스크 1의 개수: {batch['attention_mask'][1].sum().item()}")
print(f"1번 샘플 어텐션 마스크 0의 개수: {(batch['attention_mask'][1] == 0).sum().item()}")
# 결과 검증: 긴 샘플은 모든 어텐션 마스크가 1이고, 짧은 샘플은 일부만 1이어야 함

0번 데이터 길이: 227
1번 데이터 길이: 176

배치 처리 후:
입력 ID 형태: torch.Size([2, 227])
어텐션 마스크 형태: torch.Size([2, 227])
0번 샘플 어텐션 마스크 합계: 227
1번 샘플 어텐션 마스크 합계: 176

0번과 1번 샘플의 어텐션 마스크가 다른가요? True

0번 샘플 어텐션 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1번 샘플 어텐션 마스크: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## 6. 전처리 이해하기

**input_ids와 labels는 어떻게 생성되는가?**

LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 시스템 프롬프트까지 포함하여 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다:
- 시스템 프롬프트: `당신은 친절하고 도움이 되는 AI 어시스턴트입니다.`
- 사용자 메시지: `안녕하세요, 오늘 날씨는 어떤가요?`
- 어시스턴트 응답: `안녕하세요! 오늘 날씨는 맑고 화창합니다.`

LLaMA 3에서는 다음과 같은 템플릿 구조를 사용합니다(줄바꿈 포함):

```python
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 친절하고 도움이 되는 AI 어시스턴트입니다.<|eot_id|><|start_header_id|>user<|end_header_id|>
안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>
```

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환해봅시다.  
(실제와 다르고 가정하여 정수를 맵핑하겠습니다.)

먼저 모든 특수 토큰들은 아래의 고유 ID를 가진다고 가정해봅시다.  
- <|begin_of_text|> = 토큰 ID 1
- <|start_header_id|> = 토큰 ID 2
- <|end_header_id|> = 토큰 ID 4
- 줄바꿈 = 토큰 ID 5
- <|eot_id|> = 토큰 ID 10

역할 토큰들은 아래의 고유 ID를 가진다고 가정해봅시다.  
- system = 토큰 ID 3
- user = 토큰 ID 11
- assistant = 토큰 ID 18

전체 통합된 input_ids는 다음과 같습니다:
`input_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 2, 11, 4, 5, 12, 13, 14, 15, 16, 17, 10, 2, 18, 4, 5, 19, 20, 21, 22, 23, 10]`

각 부분을 분리하면:
- 시스템 프롬프트 부분: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
- 사용자 메시지 부분: [2, 11, 4, 5, 12, 13, 14, 15, 16, 17, 10]
- 어시스턴트 응답 부분: [2, 18, 4, 5, 19, 20, 21, 22, 23, 10]

모델이 예측해야 할 영역은 assistant의 응답 부분인 `안녕하세요! 오늘 날씨는 맑고 화창합니다.`에 해당하는 토큰들입니다. 따라서 `labels`는 다음과 같이 설정됩니다:

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 19, 20, 21, 22, 23, 10]`

여기서 주목할 점:
1. 시스템 프롬프트와 사용자 메시지에 해당하는 모든 토큰(줄바꿈 포함)은 `-100`으로 마스킹됩니다.
2. 어시스턴트 헤더와 첫 줄바꿈 토큰도 `-100`으로 마스킹됩니다.
3. 실제 어시스턴트 응답 내용(19-23)과 마지막 종료 태그(10)만 원래 토큰 ID를 유지합니다.

이처럼 `labels`는 모델이 실제로 생성해야 할 출력 부분만을 포함하고, 나머지 부분은 `-100`으로 채워져 손실 계산에서 제외됩니다. 이를 통해 모델은 입력(시스템 프롬프트+사용자 질문)을 기반으로 적절한 응답을 생성하는 방법을 학습합니다.

학습 과정에서는:
1. 모델에 `input_ids` 전체를 입력으로 제공합니다.
2. 모델은 각 위치에서 다음 토큰을 예측합니다.
3. 손실 계산 시 `labels`가 `-100`이 아닌 위치에서만 오차를 계산합니다.
4. 이를 통해 모델은 주어진 맥락(시스템 프롬프트와 사용자 질문)에 대해 적절한 응답을 생성하는 방법을 학습합니다.

## 7. 학습하기

In [22]:
trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


- **학습 데이터 크기**: 2500개

모델 학습에 사용되는 전체 데이터 샘플의 수입니다. 이는 각 에포크마다 처리되는 전체 데이터 양을 의미합니다.

- **에포크(epochs)**: 3회

전체 데이터셋을 처음부터 끝까지 반복해서 학습하는 횟수입니다. 즉, 모든 학습 데이터를 3번 반복해서 모델이 학습한다는 의미입니다.

- **배치 크기(batch size)**: 2

한 번에 처리하는 데이터 샘플의 수입니다. 메모리 효율을 위해 전체 데이터를 작은 배치로 나누어 처리하며, 여기서는 2개씩 묶어서 처리합니다.

- **누적 단계(accumulation steps)**: 2

모델을 실제로 업데이트하기 전에 여러 배치의 정보를 모으는 수입니다. 여기서는 2개의 배치(총 4개의 샘플)를 처리한 후에야 실제 모델 업데이트가 일어납니다.

- **에포크 1회당 업데이트 횟수**: 2500 ÷ (2 × 2) = 625회

한 에포크에서 모델이 업데이트되는 횟수입니다. 전체 데이터 2500개를 유효 배치 크기 4(배치 크기 2 × 누적 단계 2)로 나누면 625번의 업데이트가 발생합니다.

- **총 업데이트 계산 방법**: (데이터 크기 × 에포크) ÷ (배치 크기 × 누적 단계)

학습 과정 전체에서 발생하는 모델 업데이트의 총 횟수를 계산하는 공식입니다. 전체 처리 샘플 수를 유효 배치 크기로 나눕니다.

- **총 업데이트 계산 과정**: (2500 × 3) ÷ (2 × 2) = 7,500 ÷ 4 = 1,875

3개의 에포크 동안 총 7,500개의 샘플이 처리되고, 유효 배치 크기인 4개의 샘플마다 한 번씩 모델이 업데이트되므로 총 1,875번의 모델 업데이트가 발생합니다.

In [23]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.765700
20,0.288800
30,0.261200
40,0.298500
50,0.285900
60,0.290900
70,0.207100
80,0.271100
90,0.235000
100,0.188100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-pac

## 8. 테스트 데이터 준비하기

실제 모델에 입력을 넣을 때에는 입력의 뒤에 `<|start_header_id|>assistant<|end_header_id|>\n`가 부착되어서 넣는 것이 좋습니다. 그러면 모델이 조금 더 안정적으로 답변을 생성합니다.

In [24]:
prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [25]:
print(prompt_lst[200])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 text-to-sql을 수행해야 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력 텍스트: 텍사스와 캘리포니아 주에서 발급된 상업용 건물의 모든 건축 허가증 번호를 나열하세요.

DDL statements:
CREATE TABLE permit (id INT, state VARCHAR(20), type VARCHAR(20), permit_number INT); INSERT INTO permit (id, state, type, permit_number) VALUES (1, 'Texas', 'Commercial', 100), (2, 'Texas', 'Residential', 150), (3, 'California', 'Commercial', 80), (4, 'California', 'Residential', 200);

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



In [26]:
print(label_lst[200])


쿼리 작성: SELECT permit_number FROM permit WHERE (state = 'Texas' OR state = 'California') AND type = 'Commercial';


## 9. 파인튜닝 모델 테스트

`AutoPeftModelForCausalLM()`의 입력으로 LoRA Adapter가 저장된 체크포인트의 주소를 넣으면 LoRA Adapter가 기존의 LLM과 부착되어 로드됩니다. 이 과정은 LoRA Adapter의 가중치를 사전 학습된 언어 모델(LLM)에 통합하여 미세 조정된 모델을 완성하는 것을 의미합니다.

`peft_model_id` 변수는 미세 조정된 가중치가 저장된 체크포인트의 경로를 나타냅니다. `"llama3-8b-text-to-sql/checkpoint-1875"`는 LoRA Adapter 가중치가 저장된 위치로, 이 경로에서 해당 가중치를 불러옵니다.

`fine_tuned_model`은 `AutoPeftModelForCausalLM.from_pretrained` 메서드를 통해 체크포인트를 로드하여 생성됩니다. 이 메서드는 LLM과 LoRA Adapter를 결합하고, 최적화된 설정으로 모델을 메모리에 로드합니다. `device_map="auto"` 옵션은 모델을 자동으로 GPU에 배치합니다.

`pipeline`은 Hugging Face의 고수준 유틸리티로, NLP 작업(예: 텍스트 생성, 번역, 요약 등)을 간단히 수행할 수 있게 해줍니다. 이 코드에서 사용된 `pipeline("text-generation")`은 텍스트 생성 작업을 수행하기 위한 파이프라인 객체를 생성합니다. 파이프라인은 내부적으로 모델과 토크나이저를 관리하여, 입력 텍스트를 토큰화하고, 모델을 통해 생성된 결과를 다시 디코딩하여 사람이 읽을 수 있는 텍스트로 변환합니다.

이 코드는 미세 조정된 LLM을 로드한 뒤, 이를 이용해 텍스트 생성 작업을 간단히 수행할 수 있도록 준비하는 데 목적이 있습니다. `pipeline`을 통해 텍스트 생성 작업을 실행하면, 입력 텍스트에 기반하여 모델이 다음 토큰을 예측하고 이를 반복적으로 생성합니다. 이 과정은 사용자에게 자연스러운 텍스트를 출력하는 데 사용됩니다.데 사용됩니다.

In [27]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [28]:
peft_model_id = "llama3-8b-text-to-sql/checkpoint-1875"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [29]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

In [30]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [31]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


    response:
쿼리 작성: SELECT AVG(age) FROM community_engagement;
    label:

쿼리 작성: SELECT AVG(age) FROM community_engagement WHERE language IS NOT NULL;
--------------------------------------------------
    response:
쿼리 작성: SELECT AVG(price) FROM skincare_sales WHERE sale_country = 'Canada' AND product_name LIKE '%organic%';
    label:

쿼리 작성: SELECT AVG(price) FROM skincare_sales WHERE sale_country = 'Canada' AND product_name LIKE '%organic%';
--------------------------------------------------
    response:
쿼리 작성: SELECT agency.name, COUNT(event.id) as total_events FROM agency INNER JOIN event ON agency.id = event.agency_id GROUP BY agency.name;
    label:

쿼리 작성: SELECT agency_id, COUNT(*) as total_events FROM event GROUP BY agency_id;
--------------------------------------------------
    response:
쿼리 작성: SELECT country, COUNT(*) as satellites_in_orbit FROM satellites_in_orbit WHERE last_update = '2022-07-01' GROUP BY country ORDER BY satellites_in_orbit DESC;
    label:

쿼리 작성: SE

## 10. 기본 모델 테스트

이번에는 LoRA Adapter를 merge하지 않은 기본 모델로 테스트 데이터에 대해서 인퍼런스해보겠습니다.


In [32]:
base_model_id = "allganize/Llama-3-Alpha-Ko-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
for prompt, label in zip(prompt_lst[10:15], label_lst[10:15]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

    response:
입력 텍스트에서 "언어를 사용하는 사람들의 평균 나이"를 구하려면 다음과 같은 SQL 쿼리를 작성할 수 있습니다:

```sql
SELECT AVG(age) AS avg_age
FROM community_engagement
WHERE language = 'English';
```

이 쿼리는 `community_engagement` 테이블에서 `language` 열 값이 'English'인 행의 `age` 열 값을 평균으로 계산합니다.

이 쿼리를 실행하면 다음과 같은 결과를 얻을 수 있습니다:

```sql
avg_age
-------
 45.000
```

이 결과는 테이블에 있는 영어를 사용하는 사람들의 평균 나이를 나타냅니다.
    label:

쿼리 작성: SELECT AVG(age) FROM community_engagement WHERE language IS NOT NULL;
--------------------------------------------------
    response:
입력 텍스트에서 제공된 정보를 바탕으로 SQL 쿼리를 작성하려면, 평균 가격을 구하려면 테이블에 저장된 모든 데이터를 사용해야 합니다. 현재 테이블에는 단 하나의 행이 있습니다. 따라서 평균을 구할 수 없습니다. 그러나 사용자가 추가 데이터를 제공하면 평균 가격을 계산할 수 있습니다.

다음은 현재 데이터를 기반으로 작성한 SQL 쿼리입니다:

```sql
SELECT product_name, price, sale_country
FROM skincare_sales;
```

이 쿼리는 `skincare_sales` 테이블의 모든 행을 반환합니다. 현재 테이블에는 하나의 행만 있으므로, 이 쿼리는 다음과 같은 결과를 제공합니다:

| product_name       | price | sale_country |
|-------------------|-------|-------------|
| Organic Day Cream | 35.50